## Segmenting and Clustering Neighborhoods in Toronto
## Peer-Graded Assignment
## Part 1 

## Introduction

For this assignment, we were required to explore and cluster the neighborhoods in Toronto.For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto, The data given were in tabular format consisting of three columns -"Postal Code", "Borough", "Neighborhood".
The objective was to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format.
Once the data is in a structured format,we can easily explore and cluster the neighborhoods in the city of Toronto.

WIKIPEDIA LINK - "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

## Libraries Required

### BeautifulSoup
Beautiful Soup is a Python library for pulling data out of HTML and XML files. It works with your favorite parser to provide idiomatic ways of navigating, searching, and modifying the parse tree. It commonly saves programmers hours or days of work.

### Requests
Requests is a Python HTTP library, released under the Apache License 2.0. The goal of the project is to make HTTP requests simpler and more human-friendly. The current version is 2.23.0.Requests is one of the most popular python libraries that is not included with python, it has been proposed that requests be distributed with python by default.

### Pandas
pandas is a software library written for the Python programming language for data manipulation and analysis. In particular, it offers data structures and operations for manipulating numerical tables and time series. It is free software released under the three-clause BSD license.

### Numpy
Numpy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
print("Required library installed")

Required library installed


## Step 1
## Scrap data from wikipedia 

## Step 1.1
#### 1. Getting data from wikipedia
#### 2.Applying BeautifulSoup to parse html data
#### 3.Storing parsed data into Pandas DataFrame (df)

In [3]:
#get html from wiki page and create soup object
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, 'lxml')

#using soup object, iterate the .wikitable to get the data from the HTML page and store it into a list
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    #First row of data is the header
    if (index == 0):
        columns = section
    else:
        data.append(section)

#convert list into Pandas DataFrame
canada_df = pd.DataFrame(data = data,columns = columns)
canada_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Step 1.2: Data Cleanup
### According to a given conditions
1. Remove Boroughs that are 'Not assigned'
2. More than one neighborhood can exist in one postal code area, combined these into one row with the neighborhoods separated with a comma
3. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [4]:
#Remove Boroughs that are 'Not assigned'
canada_df = canada_df[canada_df['Borough'] != 'Not assigned']
canada_df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [32]:
canada_df = canada_df.groupby(["Postal Code", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
canada_df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
canada_df['Neighborhood'].replace("Not assigned", canada_df["Borough"],inplace=True)
canada_df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Step 1.3- Output
#### Applying ".shape" method

In [6]:
canada_df.shape

(103, 3)